<a href="https://colab.research.google.com/github/Jordy-Rodrigues/Classifica-o_com_-OneHotEncoder/blob/main/Dados_categoricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UTILIZANDO O OneHotEncoder E O MODELO DE MACHINE LEARNING svm

In [1]:
# IMPORTAR AS BIBLIOTECAS BÁSICAS
import pandas as pd
import numpy as np
# IMPORTAR AS BIBLIOTCAS DE MACHINE LEARNING
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
# CARREGAR O DATASET
dados = pd.read_csv('/content/drive/MyDrive/exemplo3.csv',delimiter=',')
# MOSTRAR O CABEÇALHO E AS 5 PRIMEIRAS LINHAS
dados.head()

,idade,conta_corrente,sexo,risco
0,21.257389,783.127911,masculino,ruim
1,21.766573,979.747521,feminino,bom
2,25.321033,1065.328054,feminino,bom
3,23.919268,1195.758078,feminino,bom
4,21.805298,1083.764450,feminino,bom


In [3]:
# SEPARAR AS VARIÁVEIS X E Y
x = dados.drop('risco',axis=1)
y = dados.risco

In [4]:
# MOSTRAR X
x.head()

,idade,conta_corrente,sexo
0,21.257389,783.127911,masculino
1,21.766573,979.747521,feminino
2,25.321033,1065.328054,feminino
3,23.919268,1195.758078,feminino
4,21.805298,1083.764450,feminino


In [5]:
# MOSTRAR Y
y.head()

0    ruim
1     bom
2     bom
3     bom
4     bom
Name: risco, dtype: object

In [6]:
# FAZER A SELEÇÃO DAS VARIÁVEIS DE INPUT DO TIPO OBJETO, PARÂMETRO UTILIZADO include
x_cat = x.select_dtypes(include='object')

In [7]:
# FAZER A BINARIZAÇÃO DA COLUNA DE INPUT UTILIZANDO O OneHotEncoder
# PARÂMETRO sparce=False (PARA QUE O RESULTADO NÃO SEJA GERADO NO FORMATO DE UMA MATRIZ ESPERSA)
# PARÂMETRO drop='first' (PARA ELIMINAR REDUNDÂNCIA NA BINARIZAÇÃO)
onehot = OneHotEncoder(sparse=False, drop='first')
# FAZER A TRANSFORMAÇÃO
x_bin = onehot.fit_transform(x_cat)
# MOSTRAR A COLUNA BINARIZADA
x_bin

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])

In [8]:
# SELECIONAR A COLUNA DO TIPO object E EXCLUIR 
x_num = x.select_dtypes(exclude='object')
# UTILIZAR O ALGORITMO DE NORMALIZAÇÃO
minmax = MinMaxScaler()
# FAZER A NORMALIZAÇÃO DOS DADOS NUMÉRICOS
x_num = minmax.fit_transform(x_num)
# MOSTRAR OS DADOS NORMALIZADOS
x_num

array([[0.05586473, 0.06720224],
       [0.06499436, 0.09938945],
       [0.12872564, 0.11339923],
       ...,
       [0.85424954, 0.36522222],
       [0.62401886, 0.42781034],
       [0.61473291, 0.54031447]])

In [9]:
# UNIR AS COLUNAS X_BIN E X_NUM NA NOVA VARIÁVEL X_ALL, UTILIZANDO O COMANDO numpy.append() 
x_all = np.append(x_num, x_bin, axis=1)
# MOSTRAR O DATAFRAME
x_all

array([[0.05586473, 0.06720224, 1.        ],
       [0.06499436, 0.09938945, 0.        ],
       [0.12872564, 0.11339923, 0.        ],
       ...,
       [0.85424954, 0.36522222, 1.        ],
       [0.62401886, 0.42781034, 1.        ],
       [0.61473291, 0.54031447, 1.        ]])

In [10]:
# DIVIDIR A BASE DE TREINAMENTO E DE TESTE
x_treino, x_teste, y_treino, y_teste = train_test_split(x_all, y, test_size=1/3, random_state=42 )
# IMPORTAR E FAZER O TREINAMENTO DO CLASSIFICADOR SVM
svc2 = SVC()
svc2.fit(x_treino,y_treino)
# MOSTRAR A PONTUAÇÃO QUE O ALGORITMO TEVE
accuracy_score(y_teste, svc2.predict(x_teste))

0.7146282973621103

In [11]:
# CRIAR UM DATAFRAME PARA TESTAR O ALGORITMO COM NOVOS DADOS
df = pd.DataFrame({'idade':[20,25,50,35,75], 'conta_corrente':[800,4000,2200,3200,1000], 'sexo':['masculino','feminino','masculino','feminino','feminino']})
# MOSTRAR O DATAFRAME
df

,idade,conta_corrente,sexo
0,20,800,masculino
1,25,4000,feminino
2,50,2200,masculino
3,35,3200,feminino
4,75,1000,feminino


In [12]:
# PARA FAZER A PREVISÃO VOU TRANSFORMAR O DATAFRAME PARA O MESMO FORMATO QUE FOI UTILIZADO NO TREINAMENTO 

# FAZER A SELEÇÃO DAS VARIÁVEIS DE INPUT DO TIPO OBJETO, PARÂMETRO UTILIZADO include
x_new_bin = onehot.transform(df.select_dtypes(include=['object']))
# FAZER A NORMALIZAÇÃO NO DATAFRAME EXCETO A COLUNA DO TIPO OBJETO
x_new_num = minmax.transform(df.select_dtypes(exclude=['object']))
# UNIR AS DUAS COLUNAS 
x_new = np.append(x_new_num, x_new_bin, axis=1) 
# PREVISÃO DO MODELO
svc2.predict(x_new)

array(['ruim', 'bom', 'ruim', 'bom', 'bom'], dtype=object)

In [13]:
# MOSTRAR O RESULTADO DA PREVISÃO 

# FAZER UMA COPIA DO DATAFRAME
df_previsao = df.copy()
# VAI INCLUIR UMA COLUNA COM O NOME DE Previsão E ESSA COLUNA VAI RECEBER OS DADOS QUE O MODELO FEZ A PREVISÃO
df_previsao ['Previsão'] = svc2.predict(x_new)
# MOSTRAR A TABELA
df_previsao

,idade,conta_corrente,sexo,Previsão
0,20,800,masculino,ruim
1,25,4000,feminino,bom
2,50,2200,masculino,ruim
3,35,3200,feminino,bom
4,75,1000,feminino,bom
